In [27]:
import numpy as np
import matplotlib
import json
import pandas as pd
import os
import spacy
from collections import defaultdict
from nltk.tokenize import word_tokenize


In [28]:
def get_num_tokens(text):
    # return text.split()
    # doc = nlp(text)
    # return [tok.text for tok in doc]
    return len(word_tokenize(text))


In [33]:
splits = ["train", "valid_seen", "valid_unseen", "tests_seen", "tests_unseen"]
task_fields = ["task_type", "focus_object", "base_object", "dest_object", "scene"]
stats_dict = defaultdict(lambda: [])
data_path = "../tars/alfred/data/json_2.1.0"
for split in ["valid_seen"]:
    print(split)
    for j, task_dir in enumerate(os.listdir("{}/{}".format(data_path, split))):
        if j % 500 == 0:
            print(j)
        task_values = task_dir.split("-")
        if len(task_values) < 5:
            print(task_values)
        for trial_dir in os.listdir("{}/{}/{}".format(data_path, split, task_dir)):
            stats_dict["task_id"].append(trial_dir)
            stats_dict["split"].append(split)
            for i, field in enumerate(task_fields):
                stats_dict[field].append(task_values[i])
            traj_data_file = open("{}/{}/{}/{}/traj_data.json".format(data_path, split, task_dir, trial_dir))
            traj_data = json.load(traj_data_file)
            num_steps_list = []
            num_step_tokens_list = []
            num_task_tokens_list = []
            for directive in traj_data["turk_annotations"]["anns"]:
                num_steps_list.append(len(directive["high_descs"]))
                num_step_tokens_list.append(sum([get_num_tokens(desc) for desc in directive["high_descs"]]))
                num_task_tokens_list.append(get_num_tokens(directive["task_desc"]))
            stats_dict["num_steps"].append(np.mean(num_steps_list))
            stats_dict["steps_num_toks"].append(np.mean(num_step_tokens_list))
            stats_dict["task_num_toks"].append(np.mean(num_task_tokens_list))
            stats_dict["timesteps"].append(len(traj_data["plan"]["low_actions"]))
            nav_count = 0
            interact_count = 0
            for action in traj_data["plan"]["low_actions"]:
                if "mask" in action["discrete_action"]["args"].keys():
                    interact_count += 1
                else:
                    nav_count += 1
            stats_dict["nav_count"].append(nav_count)
            stats_dict["interact_count"].append(interact_count)

stats_df = pandas.DataFrame(stats_dict)
        





  

valid_seen
0


In [34]:
stats_df

,task_id,split,task_type,focus_object,base_object,dest_object,scene,num_steps,steps_num_toks,task_num_toks,timesteps,nav_count,interact_count
0,trial_T20190909_121908_219603,valid_seen,pick_clean_then_place_in_recep,Ladle,None,CounterTop,8,6.0,67.333333,7.666667,38,32,6
1,trial_T20190906_162502_940304,valid_seen,pick_and_place_simple,Cloth,None,BathtubBasin,405,4.0,38.666667,8.666667,28,24,4
2,trial_T20190907_033843_707544,valid_seen,pick_two_obj_and_place,SprayBottle,None,Toilet,429,8.0,89.800000,7.800000,76,72,4
3,trial_T20190908_134410_550530,valid_seen,pick_and_place_with_movable_recep,TomatoSliced,Pan,Fridge,16,13.0,163.000000,13.333333,136,127,9
4,trial_T20190908_070424_700178,valid_seen,pick_heat_then_place_in_recep,BreadSliced,None,Fridge,19,12.0,110.333333,12.000000,92,75,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,trial_T20190909_082320_103350,valid_seen,pick_heat_then_place_in_recep,Tomato,None,Fridge,23,6.0,71.000000,10.000000,61,47,14
247,trial_T20190908_033721_967359,valid_seen,pick_heat_then_place_in_recep,Tomato,None,Fridge,24,6.0,76.666667,8.000000,48,36,12
248,trial_T20190906_192823_237997,valid_seen,pick_cool_then_place_in_recep,Cup,None,SideTable,21,6.0,123.000000,9.000000,58,50,8
249,trial_T20190908_124829_632660,valid_seen,pick_and_place_simple,Tomato,None,Microwave,13,4.0,57.333333,7.000000,29,25,4
